In [8]:
CHART_RELEASE_NAME='pnda'

In [9]:
import random
import time
from kafka import KafkaProducer


def openmetrics_producer(stop_event,brokers, topic, metric, rate):
    producer = KafkaProducer(bootstrap_servers=brokers.split())
    print("Starting Streaming {} to Kafka: rate {} seconds.".format(metric,rate))
    while not stop_event.wait(rate):
        openmetric = '{metric} {value:.2f}'.format(metric=metric, value=random.uniform(0,50))
        future = producer.send(
            'openmetrics',
            key=metric.encode('utf-8'),
            value= openmetric.encode('utf-8'))
        # Block for 'synchronous sends'
        record_metadata = future.get(timeout=1)

In [ ]:
import threading

brokers = '{}-cp-kafka:9092'.format(CHART_RELEASE_NAME)
topic = 'openmetrics'

jobs = []
kill_event = threading.Event()
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'temperature{"room"="bathroom"}',2]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'humidity{"room"="bathroom"}',5]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'temperature{"room"="kitchen"}',3.2]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'humidity{"room"="kitchen"}',2.3]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'temperature{"room"="corridor"}',1]))
jobs.append(threading.Thread(target=openmetrics_producer, args=[kill_event,brokers,topic,'humidity{"room"="corridor"}',4]))

list(map(lambda x: x.start(), jobs))

try:
    while True:
        pass
except KeyboardInterrupt:
    print("stopping openmetrics generators")
    kill_event.set()
    list(map(lambda x: x.join(), jobs))
 


Starting Streaming humidity{"room"="corridor"} to Kafka: rate 4 seconds.
Starting Streaming temperature{"room"="bathroom"} to Kafka: rate 2 seconds.
Starting Streaming temperature{"room"="kitchen"} to Kafka: rate 3.2 seconds.
Starting Streaming humidity{"room"="kitchen"} to Kafka: rate 2.3 seconds.Starting Streaming humidity{"room"="bathroom"} to Kafka: rate 5 seconds.

Starting Streaming temperature{"room"="corridor"} to Kafka: rate 1 seconds.
